In [1]:
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path
import re
import os
from api import GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [2]:
papers_path = Path("papers/selection")
def load_all_files(root_path, mode="page"):
    docs = []
    for path, folders, files in root_path.walk():
        if len(folders) != 0:
            continue
        for file in files:
            if not file.endswith(".pdf"):
                continue
            doc_loader = PyPDFLoader(
                path / file,
                mode=mode,
                extraction_mode="layout"
            )
            pages = doc_loader.load()
            docs.extend(pages)
        
    return docs

docs = load_all_files(papers_path)
        

Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text

In [3]:
def preprocess_documents(docs):
    for doc in docs:
        doc.page_content = re.sub(" +", " ", doc.page_content)
    
    return docs
docs = preprocess_documents(docs)

In [4]:
from langchain_core.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use around three sentences for your answer and keep it concise.
Do not ever mention that you are using the context as source.

Question: {question}

Context:
{context}

Answer:"    
"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [5]:
from langchain import hub
from langchain_chroma import Chroma
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client import models
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
client = QdrantClient(path="papers_db")
client.create_collection(
    collection_name="selection",
    vectors_config=models.VectorParams(
        size=768,
        distance=models.Distance.COSINE
    )

)
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=embeddings,
#     persist_directory="selection_db"
#  )
# vectorstore = Chroma(
#     persist_directory="selection_db",
#     embedding_function=embeddings
# )
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="selection",
    embedding=embeddings,
    distance=models.Distance.COSINE
)
vectorstore.add_documents(splits)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
prompt = custom_rag_prompt


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Use around three sentences for your answer and keep it concise."
    "Do not ever mention that you are using the context as source."
    "Here is context for your answer:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

rag_chain.invoke({"input": "What kind of material is MAPbI3?"})

{'input': 'What kind of material is MAPbI3?',
 'context': [Document(metadata={'producer': 'PDFlib+PDI 9.1.2p4 (C++/Win64); modified using iText 4.2.0 by 1T3XT', 'creator': 'Arbortext Publishing Engine', 'creationdate': '2024-12-31T14:36:36-05:00', 'moddate': '2025-03-26T08:25:41-07:00', 'fpage': '184', 'acs openaccess version': 'v3.2', 'subject': 'J. Phys. Chem. Lett. 2025.16:184-190', 'printengine': 'Arbortext Advanced Print Publisher 11.2.5208/W Library-x64', 'author': 'Pelayo Marin-Villa, Mattia Gaboardi, Boby Joseph, Frederico Alabarse, Jeff Armstrong, Kacper Drużbicki, and Felix Fernandez-Alonso', 'title': 'Methylammonium Lead Iodide across Physical Space: Phase Boundaries and Structural Collapse', 'epsprocessor': 'PStill version 1.84.42', 'prism:doi': '10.1021/acs.jpclett.4c03336', 'lpage': '190', 'prism:versionidentifier': 'Issue', 'jcalibrarybuild': '5.2.i3', 'acs openaccess stamp': '01/02/2025 15:17:17 (CCBY4_0)', 'jav:journal_article_version': 'VoR', 'source': 'papers/selecti

In [6]:
rag_chain.invoke({"input": "What is the spin space group symmetry of Fe3GeTe2?"})

{'input': 'What is the spin space group symmetry of Fe3GeTe2?',
 'context': [Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-02-12T01:42:39+00:00', 'author': '', 'keywords': '', 'moddate': '2025-02-12T01:42:39+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'papers/selection/6_fgt.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1', '_id': 'c45d8848f32845dd97c60641a565553b', '_collection_name': 'selection'}, page_content='I. INTRODUCTION coupling(SOC)hasanegligibleeffect. Thenon-zerolocal\n magnetic moments break time reversal symmetry (TRS)\n Fe 3GeTe 2 is an itinerant ferromagnet with a Curie intrinsically, a necessary condition for some specific fea-\ntemperature T c ≈ 220 K and a hexagonal layered tures to appear. For example, centrosymmetric crystals\nstructure[1]. The bulk material is formed by s